In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
# from src.inference import get_feature_store
# from datetime import datetime, timedelta
# import pandas as pd  

# # Get the current datetime64[us, Etc/UTC]  
# current_date = pd.Timestamp.now(tz='Etc/UTC')
# feature_store = get_feature_store()

# # read time-series data from the feature store
# fetch_data_to = current_date - timedelta(hours=1)
# fetch_data_from = current_date - timedelta(days=1*29)
# print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
# feature_view = feature_store.get_feature_view(
#     name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
# )

# ts_data = feature_view.get_batch_data(
#     start_time=(fetch_data_from - timedelta(days=1)),
#     end_time=(fetch_data_to + timedelta(days=1)),
# )
# ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
# ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
# ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

# from src.data_utils import transform_ts_data_info_features
# features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)







from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
    write_options={"use_spark": True}  # Bypass ArrowFlight
)

ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-05-27 00:06:17,155 INFO: Initializing external client
2025-05-27 00:06:17,157 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:06:19,860 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057
Fetching data from 2025-04-28 04:06:17.149722+00:00 to 2025-05-27 03:06:17.149722+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.52s) 


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-27 00:06:43,576 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-27 00:06:43,588 INFO: Initializing external client
2025-05-27 00:06:43,589 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:06:45,782 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057


Downloading: 0.000%|          | 0/313830 elapsed<00:00 remaining<?

In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [7]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,263,112.0,2025-05-27 05:00:00+00:00
1,25,2.0,2025-05-27 05:00:00+00:00
2,150,0.0,2025-05-27 05:00:00+00:00
3,56,0.0,2025-05-27 05:00:00+00:00
4,32,0.0,2025-05-27 05:00:00+00:00
...,...,...,...
250,180,0.0,2025-05-27 05:00:00+00:00
251,27,0.0,2025-05-27 05:00:00+00:00
252,126,0.0,2025-05-27 05:00:00+00:00
253,182,1.0,2025-05-27 05:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-05-27 00:06:52,597 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-27 00:06:52,607 INFO: Initializing external client
2025-05-27 00:06:52,609 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:06:54,585 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1233057/fs/1216592/fg/1479123


Uploading Dataframe: 100.00% |█████████████████████████████| Rows 255/255 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1233057/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)